In [9]:
import json
from tqdm import tqdm
import pandas as pd
import numpy as np

In [10]:
def get_tags(source, target_r, target_a):
    output_r = open(target_r, 'w', encoding='utf-8')
    output_a = open(target_a, 'w', encoding='utf-8')
    with open(source, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in tqdm(data):
            # id = item['id']
            # text = item['sentence']
            words = item['sentence']

            # words = list(text)
            # words = list(text.replace(' ', ''))
            # words = get_buffer(text)
            triggers = ['O'] * len(words)
            entities = [['O'] for _ in range(len(words))]
            arguments = ['O'] * len(words)


            # triggers
            for event_mention in item['golden-event-mentions']:
                for i in range(event_mention['trigger']['start'], event_mention['trigger']['end']):
                    trigger_type = event_mention['event_type']
                    if i == event_mention['trigger']['start']:
                        triggers[i] = 'B-{}'.format(trigger_type)
                    else:
                        triggers[i] = 'I-{}'.format(trigger_type)

                for argument in event_mention['arguments']:
                    role = argument['role']
                    # if role.startswith('Time'):
                    #     role = role.split('-')[0]
                    for j in range(argument['start'], argument['end']):
                        if j == argument['start']:
                            arguments[j] = 'B-{}'.format(role)
                        else:
                            arguments[j] = 'I-{}'.format(role)

                # print(arguments, len(arguments))
                # 获得事件文件
                for w_, t_ in zip(words, triggers):
                    if w_.strip() == '':
                        continue
                    output_r.write('{}\t{}\n'.format(w_, t_))
                output_r.write('\n')

                # 获得role文件
                for w_, a_ in zip(words, arguments):
                    if w_.strip() == '':
                        continue
                    output_a.write('{}\t{}\n'.format(w_, a_))
                output_a.write('\n')

In [16]:
get_tags('../data/chinese_relation_ace05/train.json', 'trigger_train.txt', 'argument_train.txt')

JSONDecodeError: Extra data: line 2 column 1 (char 418)